In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import log, sqrt, exp
from scipy import stats
from math import log, sqrt, exp
from scipy import stats
from scipy.stats import norm
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statistics import NormalDist

class BSM:
#class constructor; input: S,vol,r,q
    def __init__(self,S0,vol,r,q):
        """class constructor; input: S0,vol,r,q"""
        self.S, self.vol, self.r, self.q = S0, vol, r, q

# alternative constructor; take data from dictionary
    @classmethod
    def from_dict( cls_, d ):
        S=d['S'] if 'S' in d else 100
        vol=d['vol'] if 'vol' in d else 0.3
        r=d['r'] if 'r' in d else 0
        q=d['q'] if 'q' in d else 0
        return cls_(S,vol,r,q)
    
    def discount(self, T):
        return np.exp(-(self.r) * T)
    
    def Binary(self, K, T):
        return (1 - self.Pr(K, T)) * 1 * self.discount(T) 
    
    def Bond(self, K, T):
        return 1 * self.discount(T) 
    
    def Call(self,K,T):
        """Calculate the price of Call Option. Inputs: Strike, Time to maturity."""
        d2=(np.log(self.S / K) + (self.r-self.q - self.vol**2 / 2) * T) / (self.vol * np.sqrt(T));
        d1=(np.log(self.S/K) + (self.r -self.q+ self.vol**2 / 2) * T)/(self.vol * np.sqrt(T))
        return np.exp(-self.q * T) *self.S * norm.cdf(d1) - K * np.exp(-self.r * T) * norm.cdf(d2)

    def Put(self,K,T):
        """Calculate the price of Put Option. Inputs: Strike, Time to maturity."""
        d2=(np.log(self.S / K) + (self.r-self.q - self.vol**2 / 2) * T) / (self.vol * np.sqrt(T));
        d1=(np.log(self.S/K) + (self.r -self.q+ self.vol**2 / 2) * T)/(self.vol * np.sqrt(T))
        return -np.exp(-self.q * T) *self.S * norm.cdf(-d1) + K * np.exp(-self.r * T) * norm.cdf(-d2)

    
    def DeltaCall(self,K,T):
        """Calculate the Delta of a Call Option. Inputs: Strike, Time to maturity."""
        d1=(np.log(self.S/K) + (self.r -self.q+ self.vol**2 / 2) * T)/(self.vol * np.sqrt(T))
        return np.exp(-self.q * T) * norm.cdf(d1)

    def DeltaPut(self,K,T):
        """Calculate the Delta of a Put Option. Inputs: Strike, Time to maturity."""
        d1=(np.log(self.S/K) + (self.r -self.q+ self.vol**2 / 2) * T)/(self.vol * np.sqrt(T))
        return np.exp(-self.q * T) * (norm.cdf(d1)-1)
    

    def Gamma(self,K,T):
        """Calculate the Gamma of an Option. Inputs: Strike, Time to maturity."""
        d1=(np.log(self.S/K) + (self.r -self.q+ self.vol**2 / 2) * T)/(self.vol * np.sqrt(T))
        return norm.pdf(d1)/(self.S*self.vol*np.sqrt(T))


    def Pr(self,K,T):
        """Calculate the probability of S(T)<K. """
        d2=(np.log(self.S / K) + (self.r-self.q - self.vol**2 / 2) * T) / (self.vol * np.sqrt(T));
        return 1-norm.cdf(d2)


    def Vega(self,K,T):
        """Calculate the vega of Call option. """
        d1=(np.log(self.S/K) + (self.r -self.q+ self.vol**2 / 2) * T)/(self.vol * np.sqrt(T))
        vega = np.exp(-self.q * T)*self.S*np.sqrt(T)*norm.pdf(d1);
        return vega

In [240]:
# 1
T_1, S_2, T_2, S_3 = 2, 0.95, 3, 0.8
T_2_5 = 2.5
# lambda - ?
# S_2_5 - ?
print('1)')
print('Воспользуемся формулой S_3 = S_1*exp(-lambda(T_2-T_1)), из которой выразим lambda:')
lambda_hazard_rate = np.log(S_3/S_2)/(T_1-T_2)
print('lambda_hazard_rate = {0:.4f}'.format(lambda_hazard_rate), 'i.e. {0:.2f}'.format(lambda_hazard_rate*100),'%.')
print('')
print('2)')
S_2_5 = S_2*np.exp(-lambda_hazard_rate*(T_2_5-T_1))
print('S_2_5 = {0:.4f}'.format(S_2_5), 'i.e. {0:.2f}'.format(S_2_5*100),'%.')

1)
Воспользуемся формулой S_3 = S_1*exp(-lambda(T_2-T_1)), из которой выразим lambda:
lambda_hazard_rate = 0.1719 i.e. 17.19 %.

2)
S_2_5 = 0.8718 i.e. 87.18 %.


In [241]:
# 2
s_1 = 0.5 # USDRUB, доллар-рубль
s_2 = 0.9 # SSRRUB, акции рубль (Мосбиржа)
s_3 = 0.8 # SSSUSD, акции доллар (РТС)
# SSSUSD = SSSRUB / USDRUB
print('Воспользуемся формулой corr = ((s_2)^2 + (s_1)^2 - (s_3)^2)/2*s_2*s_1, тогда')
corr = ((s_2)**2 + (s_1)**2 - (s_3)**2)/(2*s_2*s_1)
print('Correlation = {0:.4f}'.format(corr))

Воспользуемся формулой corr = ((s_2)^2 + (s_1)^2 - (s_3)^2)/2*s_2*s_1, тогда
Correlation = 0.4667


In [242]:
# 3
n, mu, sigma = 10000, 3000, 5000 
print('1)')
print('Доверительный интервал (mu - 1.96*(sigma/sqrt(n)) ; mu - 1.96*(sigma/sqrt(n))), то есть:')
a = mu - 1.96*(sigma/np.sqrt(n))
b = mu + 1.96*(sigma/np.sqrt(n))
print(f'({a}, {b})')
print('')
print('2)')
print('Из уравнения 2*1.96*(sigma/sqrt(x)) = 20 выразим x, x = (2*1.96*sigma/20)^2, то есть:')
x = (2*1.96*sigma/20)**2
print(f'Нужно сделать {x} симуляций.')

1)
Доверительный интервал (mu - 1.96*(sigma/sqrt(n)) ; mu - 1.96*(sigma/sqrt(n))), то есть:
(2902.0, 3098.0)

2)
Из уравнения 2*1.96*(sigma/sqrt(x)) = 20 выразим x, x = (2*1.96*sigma/20)^2, то есть:
Нужно сделать 960400.0 симуляций.


In [243]:
# 4
S_0, sigma, r, K, T, div = 100, 0.3, 0.02, 102, 1, 0
m, s = 0, 0.02
# 1% VaR - ?
# y = mu + x*sigma, y ~ N(m, sigma^2), x ~ N(0, 1) 
A = BSM(S_0, sigma, r, div)
Call_1 = A.Call(K, T)
print('Call_1 = {0:.4f}'.format(Call_1))
# Ф_mu_r^2 (x) = Ф_0_1 ((x-mu)/r)
# (x-mu)/r = -2.33 => x = -2.33*r + mu
x = -2.33*r
S_1 = S_0*(1+x)
B = BSM(S_1, sigma, r, div)
Call_2 = B.Call(K, T)
print('Call_2 = {0:.4f}'.format(Call_2))
VaR = Call_1 - Call_2
print('1% VaR = {0:.4f}'.format(VaR), 'i.e. {0:.2f}'.format(VaR),'USD.')

Call_1 = 11.9322
Call_2 = 9.4691
1% VaR = 2.4631 i.e. 2.46 USD.


In [247]:
# 5
T, g = 5, 0
c, R = 0.06, 0.4
# p - ? (вероятность дефолта)
# l_h_r - ?
# fair_c - ? (CDS)

print('1)')
print('Воспользуемся формулой p = (1 - (1+r_g)^T/(1+r_c)^T) / (1-R), тогда')
p = (1 - (1 + g)**T/(1 + c)**T)/(1-R)
print('Вероятность дефолта p = {0:.4f}'.format(p))
print('Вероятность не дефолта 1-p = {0:.4f}'.format(1-p))
print('')
print('2)')
print('Воспользуемся формулой 1-p = e^(-lambda*5), откуда')
l_h_r = -np.log(1-p)/5
print('Hazard rate = {0:.4f}'.format(l_h_r), 'i.e. {0:.2f}'.format(l_h_r*100),'%.')
print('')
print('3)')
fair_c = ((1-R)*(1-np.exp(-5*l_h_r)))/(np.exp(-5*l_h_r) + np.exp(-4*l_h_r) + np.exp(-3*l_h_r) + np.exp(-2*l_h_r) + np.exp(-1*l_h_r))
print('Справедливый купон = {0:.4f}'.format(fair_c), 'i.e. {0:.2f}'.format(fair_c*100),'%.')
print('')
print('4)')
print('Воспользуемся вторым способом из лекции и сравним результаты,')
print('e^(-5*lambda) = 1 - 5*lambda, тогда')
p_s = 1 - 5*l_h_r 
print('Референсный эмитент доживёт до 5 года с вероятностью = {0:.4f}'.format(p_s))
print('Вероятность, что произойдет дефолт = {0:.4f}'.format(1-p_s))
fair_c_2 = (1-p_s)*(1-R)/5
print('Справедливый купон (2 способ) = {0:.4f}'.format(fair_c_2), 'i.e. {0:.2f}'.format(fair_c_2*100),'%.')

1)
Воспользуемся формулой p = (1 - (1+r_g)^T/(1+r_c)^T) / (1-R), тогда
Вероятность дефолта p = 0.4212
Вероятность не дефолта 1-p = 0.5788

2)
Воспользуемся формулой 1-p = e^(-lambda*5), откуда
Hazard rate = 0.1094 i.e. 10.94 %.

3)
Справедливый купон = 0.0693 i.e. 6.93 %.

4)
Воспользуемся вторым способом из лекции и сравним результаты,
e^(-5*lambda) = 1 - 5*lambda, тогда
Референсный эмитент доживёт до 5 года с вероятностью = 0.4531
Вероятность, что произойдет дефолт = 0.5469
Справедливый купон (2 способ) = 0.0656 i.e. 6.56 %.


In [248]:
# 6
T, g = 5, 0
c, R = 0.015, 0.6

# p - ? (вероятность дефолта)
# l_h_r - ?
# fair_c - ? (CDS)

print('1)')
print('Воспользуемся формулой p = (1 - (1+r_g)^T/(1+r_c)^T) / (1-R), тогда')
p = (1 - (1 + g)**T/(1 + c)**T)/(1-R)
print('Вероятность дефолта p = {0:.4f}'.format(p))
print('Вероятность не дефолта 1-p = {0:.4f}'.format(1-p))
print('')
print('2)')
print('Воспользуемся формулой 1-p = e^(-lambda*5), откуда')
l_h_r = -np.log(1-p)/5
print('Hazard rate = {0:.4f}'.format(l_h_r), 'i.e. {0:.2f}'.format(l_h_r*100),'%.')
print('')
print('3)')
print('Пусть продавец заплатит x % номинала, тогда')
print('1/100 * (e^(-1*lambda)+...+e^(-5*lambda)) + x = (1-R)*(1-e^(-5*lambda)), откуда')
x = (1-R)*(1-np.exp(-5*l_h_r))-(np.exp(-5*l_h_r) + np.exp(-4*l_h_r) + np.exp(-3*l_h_r) + np.exp(-2*l_h_r) + np.exp(-1*l_h_r))/100
print('Единоразовая выплата от номинала = {0:.4f}'.format(x), 'i.e. {0:.2f}'.format(x*100),'%.')

1)
Воспользуемся формулой p = (1 - (1+r_g)^T/(1+r_c)^T) / (1-R), тогда
Вероятность дефолта p = 0.1793
Вероятность не дефолта 1-p = 0.8207

2)
Воспользуемся формулой 1-p = e^(-lambda*5), откуда
Hazard rate = 0.0395 i.e. 3.95 %.

3)
Пусть продавец заплатит x % номинала, тогда
1/100 * (e^(-1*lambda)+...+e^(-5*lambda)) + x = (1-R)*(1-e^(-5*lambda)), откуда
Единоразовая выплата от номинала = 0.0273 i.e. 2.73 %.


In [249]:
# 7
import itertools

A, S_EUR, S_USD = 10000, 110, 100
mu, sigma_EUR, sigma_USD = 0, 0.05, 0.04
corr = 0.8
print('1)')
A_RUB = A*S_EUR
print('Текущий портфель в рублях = {0:.2f}'.format(A_RUB))
VaR_1_EUR_p = -2.33*sigma_EUR
Var_1_RUB = np.abs(VaR_1_EUR_p*A_RUB)
print('Текущий однодневный 1% VaR = {0:.2f}'.format(Var_1_RUB))
print('')
print('2)')
print('Наш новый портфель состоит из 10 000 EUR + x USD (в конце получим x со знаком), тогда')

for x in range(-10000, 10000):
    
    x -= 0.8
    # epsilon = 0.000000000000000000000001
    a = 10000*110/(10000*110 + x*100)
    b = x*100/(10000*110 + x*100)
    #print(f'вес EUR = {a}, вес USD = {b}.')
    #print('')
    #print('Далее найдем вариацию составного портфеля:')
    Var_new =  (a**2)*(sigma_EUR**2) + (b**2)*(sigma_USD**2) + 2*a*b*corr*sigma_EUR*sigma_USD
    #print(f'Var_new = {Var_new}, тогда')
    #print('')
    sigma_new = np.sqrt(Var_new)
    #print(f'{sigma_new}, тогда')
    #print('')
    VaR_1_new_p = 2.33*sigma_new
    #print(f'VaR_1_new_p = 2.33*sigma_new, т.е. {VaR_1_new_p}')
    res = VaR_1_new_p*(10000*110 + x*100) 
    
    if int(res) == 100000: #> 100000-epsilon and int(res) < 100000+epsilon:
        print(f'{x}, то есть шортим {-x} USD.')     

1)
Текущий портфель в рублях = 1100000.00
Текущий однодневный 1% VaR = 128150.00

2)
Наш новый портфель состоит из 10 000 EUR + x USD (в конце получим x со знаком), тогда
-4139.8, то есть шортим 4139.8 USD.
